# New york taxi data web scraping

In [2]:
import pandas as pd
print(pd.__version__)

1.5.2


In [4]:
df = pd.read_json("https://data.smcgov.org/resource/mb6a-xn89.json")

In [7]:
df.describe()

,less_than_high_school_graduate,high_school_graduate,some_college_or_associate_s_degree,bachelor_s_degree_or_higher,:@computed_region_uph5_8hpn,:@computed_region_i2t2_cryp
count,32.00000,32.000000,32.000000,32.000000,30.000000,32.000000
mean,17.80000,6.462500,5.946875,2.856250,17.733333,25062.093750
std,19.29944,4.693905,4.728430,1.873919,9.762466,9502.711577
min,0.00000,0.000000,0.000000,0.000000,1.000000,312.000000
25%,6.82500,1.925000,2.525000,2.100000,9.500000,28587.750000
50%,13.90000,7.750000,5.500000,3.000000,18.500000,28595.000000
75%,20.97500,9.450000,8.800000,3.600000,25.750000,28604.250000
max,100.00000,16.400000,18.500000,9.100000,34.000000,28613.000000
